<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/customer_golden_record_identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv
!pip install openai==0.28

In [2]:
import os
import openai
import pandas as pd
from dotenv import load_dotenv
from operator import itemgetter
import json
import requests
from cryptography.fernet import Fernet
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.environ['GDRIVE_CONFIG_DIR'] = "/content/drive/MyDrive"

Mounted at /content/drive


In [3]:
!ls -ltr /content/drive/MyDrive/openai_api_key.txt

-rw------- 1 root root 74 May 29 14:55 /content/drive/MyDrive/openai_api_key.txt


In [4]:
!rm -rf .env
!touch .env
!cat /content/drive/MyDrive/openai_api_key.txt >> .env

In [5]:
if not os.path.exists(".env"):
    print("Error: .env file not found. Please create a .env file and set the OPENAI_API_KEY environment variable.")
    exit(1)

In [6]:
MODEL = "gpt-3.5-turbo"
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

In [7]:
!ls -ltr /content/na_cust_names.csv

-rw-r--r-- 1 root root 71499920 Sep  3 13:10 /content/na_cust_names.csv


In [8]:
df = pd.read_csv("na_cust_names.csv")
df.shape

(1517517, 5)

In [9]:
df.head()

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD
0,SAPC11,0009106473,"GAMBLE, CARL",78749,US
1,SAPC11,0004146293,"MORALES, MIGUEL",97230,US
2,SAPC11,0009284944,BELFLOWER JOHN,78163,US
3,SAPC11,0002528182,BARROWS HARDWARE,01603,US
4,SAPC11,0001405397,<DI>EL PASO COMMUNITY COLLEGE,79925,US


In [10]:
df.tail()

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD
1517512,INFORXA,13366,WALMART WALMART 1026,47421,US
1517513,INFORXA,1092,REYNOLDS ACE HDWE,36535,US
1517514,INFORXA,5230,NEIGHBORHOOD MARKET 2411,37096,US
1517515,INFORXA,5230,WALMART WALMART 3927,28645,US
1517516,INFORXA,5230,WALMART 1721,52246,US


In [11]:
delimiter = '#'

In [26]:
system_message = f"""
You will be provided with customer names, delimited by {delimiter}. \
The name can either be a person's name or retail company name or blank or empty string.\
if the name is a retail company name, it can be nestled in the string.\
If the name is a person's name, output the name as is. \
If the name is a retail company name, extract the company name only without any number \
or special character or place name and output the company name.\
If the name is blank or empty string, then output 'No name found.'
Output has to strictly follow the instruction.
"""

In [13]:
def get_completion_from_messages(messages, model=MODEL,
 temperature=0, max_tokens=500):
 response = openai.ChatCompletion.create(
   model=model,
   messages=messages,
   temperature=temperature,
   max_tokens=max_tokens,
 )
 return response.choices[0].message["content"]

In [ ]:
refined_name_list = []
for name in df['XFRM_CUST_NAME'].values.tolist():
    user_message_1 = f"""
     tell me the refined name for {name} """
    messages =  [
    {'role':'system',
     'content': system_message},
    {'role':'user',
     'content': f"{delimiter}{user_message_1}{delimiter}"},
    ]
    response = get_completion_from_messages(messages)
    refined_name_list.append((name, response))

In [ ]:
refined_name_list[:20]

In [ ]:
[e[1] for e in refined_name_list][:20]

In [ ]:
df['REFINED_XFRM_CUST_NAME'] = [e[1] for e in refined_name_list]
df.head()

In [ ]:
df.tail()